# KoSTS 데이터를 이용하에 유사도 모델 제작
## 데이터 가져와서 필요한 내용 확인

In [79]:
!git clone https://github.com/kakaobrain/kor-nlu-datasets.git

fatal: destination path 'kor-nlu-datasets' already exists and is not an empty directory.


In [80]:
import pandas as pd

file_path = 'kor-nlu-datasets/KorSTS/sts-train.tsv'

df = pd.read_csv(file_path, sep='\t', usecols =['score', 'sentence1', 'sentence2'])
df

,score,sentence1,sentence2
0,5.00,비행기가 이륙하고 있다.,비행기가 이륙하고 있다.
1,3.80,한 남자가 큰 플루트를 연주하고 있다.,남자가 플루트를 연주하고 있다.
2,3.80,한 남자가 피자에 치즈를 뿌려놓고 있다.,한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.
3,2.60,세 남자가 체스를 하고 있다.,두 남자가 체스를 하고 있다.
4,4.25,한 남자가 첼로를 연주하고 있다.,자리에 앉은 남자가 첼로를 연주하고 있다.
...,...,...,...
5698,0.00,폭풍우 클로다흐가 영국을 강타하면서 심한 강풍,메르켈은 나토와 라트비아의 연대를 약속한다.
5699,0.00,리비아 테러리스트들이 공습에 대한 복수로 찍은 수십 명의 이집트 인질들,나일강에서 더 많은 시체가 발견되면서 이집트 보트 충돌 사고 사망자 수가 증가한다.
5700,0.00,바레인으로 향하는 대통령,시 주석 : 에볼라 퇴치를 계속 돕기 위한 중국
5701,0.00,"중국, 인도는 양국 관계를 증진시키겠다고 맹세한다",중국은 불안한 주식 거래자들을 안심시키기 위해 뒤뚱거리고 있다.


In [81]:
!pip install tensorflow

### 필요한 데이터 추출

In [82]:
from sklearn.model_selection import train_test_split
x = df.drop(columns=['score'])
y = df['score']

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=42)

In [49]:
!pip install konlpy

## 토큰화 및 숫자 변환

In [85]:
from konlpy.tag import Okt
import tensorflow as tf
def tokenize_korean(text):
  okt = Okt()
  if pd.notna(text):  # NaN 값이 아닌 경우에만 토큰화 수행
        tokens = okt.morphs(text)
        return ' '.join(tokens)
  else:
      return ''  # NaN 값이면 빈 문자열 반환

train_x['sentence1'] = train_x['sentence1'].apply(tokenize_korean)
train_x['sentence2'] = train_x['sentence2'].apply(tokenize_korean)
test_x['sentence1'] = test_x['sentence1'].apply(tokenize_korean)
test_x['sentence2'] = test_x['sentence2'].apply(tokenize_korean)

tokenizer = tf.keras.layers.TextVectorization(output_sequence_length=64)  # max_seq_length를 적절한 값으로 설정
tokenizer.adapt(train_x['sentence1'].values)
def tokenize_and_vectorize(text):
    text = tf.expand_dims(text, -1)
    return tokenizer(text)

# 데이터를 토큰화하고 숫자로 변환
train_x_tensor = tokenize_and_vectorize(train_x['sentence1'].values)
train_y_tensor = tf.constant(train_y.values, dtype=tf.float32)
test_x_tensor = tokenize_and_vectorize(test_x['sentence1'].values)
test_y_tensor = tf.constant(test_y.values, dtype=tf.float32)

# 데이터셋 생성
batch_size = 5
train_dataset = tf.data.Dataset.from_tensor_slices((train_x_tensor, train_y_tensor)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x_tensor, test_y_tensor)).batch(batch_size)

## 모델 구조 생성

In [111]:
import tensorflow as tf

#model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Dense(10, input_dim=64, activation='sigmoid'))  # input_dim을 적절한 값으로 수정 (임베딩 차원인 64로 가정)
#model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
#model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 이진 분류 문제이므로 활성화 함수를 sigmoid로 변경
input_1 = tf.keras.layers.Input(shape=(64,))
input_2 = tf.keras.layers.Input(shape=(64,))
merged_input = tf.keras.layers.Concatenate()([input_1, input_2])
dense_layer_1 = tf.keras.layers.Dense(32, activation='relu')(merged_input)
dense_layer_2 = tf.keras.layers.Dense(16, activation='relu')(dense_layer_1)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense_layer_2)
model = tf.keras.models.Model(inputs=[input_1, input_2], outputs=output)
# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # binary_crossentropy를 사용
              metrics=['accuracy'])  # accuracy 메트릭 사용

# 모델 요약
model.summary()

# 훈련
history = model.fit([train_x_tensor,train_x_tensor], train_y_tensor, validation_split=0.25, epochs=10, verbose=1)



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 128)          0           ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 dense_42 (Dense)               (None, 32)           4128        ['concatenate_1[0][0]']    

## 예측

In [97]:
predictions = model.predict(test_x_tensor)
print(predictions)

54/54 [==============================] - 0s 2ms/step
[[0.99993193]
 [0.99992657]
 [0.99993193]
 ...
 [0.99993324]
 [0.99992776]
 [0.99992085]]


In [109]:
scaled_predictions = predictions * 5.0
print(test_y_tensor[:10])
print(scaled_predictions[:10])

tf.Tensor([3.   4.4  4.4  4.25 2.   0.8  2.   5.   1.4  3.4 ], shape=(10,), dtype=float32)
[[4.9996595]
 [4.999633 ]
 [4.9996595]
 [4.9996414]
 [4.9996595]
 [4.9996276]
 [4.9996514]
 [4.9995975]
 [4.999633 ]
 [4.99964  ]]
